In [1]:
#loading some stuff 
import matplotlib.pyplot as plt
import matplotlib
import numpy as np
import pandas as pd
import pickle
import datetime


In [2]:
#setting system path
import sys
sys.path.append('/home/tobiasz820/sws_machine_learning/')
base_path = '/home/tobiasz820/data/'

In [3]:
#now loading all the necessary data files
#now loading the pickle file containing the audiogram data
with open(base_path + "AllReturns.pkl", "rb") as input_file:
    RET = pickle.load(input_file)
print("Read return data from pickle file")

with open(base_path + "LatestRFTSetting.pkl", "rb") as input_file:
    RFT = pickle.load(input_file)
print("Read RFT from pickle file")

with open(base_path + "sub_ADL.pkl", "rb") as input_file:
    sub_ADL = pickle.load(input_file)
print("Read use-time from pickle file")


FileNotFoundError: [Errno 2] No such file or directory: '/home/tobiasz820/data/AllReturns.pkl'

In [4]:
RFT_Setting = RFT[['SerialNumber','HearingInstrumentId']]
RET_basis = RET[['SerialNumber','RootCause', 'ReturnDate']]

NameError: name 'RFT' is not defined

In [5]:
sub_ADL.rename(columns={'ADLRawData_HearingInstrumentId':'HearingInstrumentId'}, inplace = True)

NameError: name 'sub_ADL' is not defined

In [6]:
#now try to merge the dataframes 
tmp = RFT_Setting.merge(RET_basis,on='SerialNumber',how='inner')
use_time_returns = pd.merge(sub_ADL,tmp.reset_index(), on ='HearingInstrumentId', how='inner')

NameError: name 'RFT_Setting' is not defined

In [7]:
cleaned_df = []
cnt = 0
hi_id_return = []
for hi_id, group in use_time_returns.groupby('HearingInstrumentId'):
    if(cnt%1000 == 0):
        print(cnt)
    cleaned_df.append(group.drop_duplicates(subset='ADLRawData_CreationTime'))
    hi_id_return.append(hi_id)
    cnt += 1
    
cleaned_df = pd.concat(cleaned_df)    

NameError: name 'use_time_returns' is not defined

In [8]:
cleaned_df.set_index(cleaned_df['ADLRawData_CreationTime'],inplace=True)

AttributeError: 'list' object has no attribute 'set_index'

In [9]:
sub_ADL['class'] = 0
sub_ADL.loc[sub_ADL['HearingInstrumentId'].isin(hi_id_return), 'class'] = 1

NameError: name 'sub_ADL' is not defined

In [10]:
#calculating time between last ADL and return
diff_days = []
hi_id_last_ADL = []

cnt = 0

for hi_id, group in cleaned_df.groupby('HearingInstrumentId'):        
        if(cnt%1000 == 0):
            print(cnt)            
        
        cnt += 1
        
        re_date = group['ReturnDate'].iloc[0]
        
        if (group.iloc[group.index < re_date].empty):
            continue
        
        diff_days.append((pd.Timestamp(re_date) - \
        pd.Timestamp(group.iloc[group.index < re_date].index[-1][:-6])).days)            
        
        if (diff_days[-1] > 30):
            continue
                
        hi_id_last_ADL.append(hi_id)      
        

AttributeError: 'list' object has no attribute 'groupby'

In [11]:
 len(diff_days)

0

In [12]:
#now plot the day difference histogram 
fig = plt.figure(figsize = (10,5))
ax = fig.gca()

plt.hist(diff_days, bins= 60)
ax.set_yticks([0, 100, 200, 300, 400, 500, 600, 700])
ax.set_yticklabels([round(x / len(diff_days) *100) for x in [0, 100, 200, 300, 400, 500, 600, 700]])


ZeroDivisionError: division by zero

In [13]:
import datetime
import random

def random_date(start, end):
    """Generate a random datetime between `start` and `end`"""
    return start + datetime.timedelta(
        # Get a random amount of seconds between `start` and `end`
        seconds=random.randint(0, int((end - start).total_seconds())),
    )

In [14]:
sub_ADL['VolumeControl']=(sub_ADL['ADLProperties_MeanVolumeControlLevelPerPreset_0'] +
sub_ADL['ADLProperties_MeanVolumeControlLevelPerPreset_1'] +
sub_ADL['ADLProperties_MeanVolumeControlLevelPerPreset_2'] +
sub_ADL['ADLProperties_MeanVolumeControlLevelPerPreset_3'] +
sub_ADL['ADLProperties_MeanVolumeControlLevelPerPreset_4'] +
sub_ADL['ADLProperties_MeanVolumeControlLevelPerPreset_5'] +
sub_ADL['ADLProperties_MeanVolumeControlLevelPerPreset_6'] +
sub_ADL['ADLProperties_MeanVolumeControlLevelPerPreset_7'])/ 8

NameError: name 'sub_ADL' is not defined

In [15]:
#now do the filtering of data before return
from random import randrange

keys = ['ADLProperties_TotalUseTime','ADLProperties_NumberOfDeviceBoots',\
        'ADLProperties_NumberOfPowerDowns','ADLProperties_NumberOfPresetSwitches',\
        'VolumeControl','HearingInstrumentId']

sub_sub_ADL = sub_ADL[keys]

cnt = 0
prereturn = []

for hi_id, group in sub_sub_ADL.groupby('HearingInstrumentId'):        
        if(cnt%1000 == 0):
            print(cnt)
            
        if (hi_id in hi_id_last_ADL):  
            
            tmp = cleaned_df[cleaned_df['HearingInstrumentId'] == hi_id]
            re_date = tmp['ReturnDate'].iloc[0]            
                                                            
            idx = group.index < re_date
            
            time_series = group[idx]                         
            
            prereturn.append(time_series)            
          
        else:
                        
            time_series = group #.iloc[0:randrange(group.shape[0])]                         
            
            prereturn.append(time_series) 
                     
        cnt += 1        

for_resample_df =pd.concat(prereturn)


NameError: name 'sub_ADL' is not defined

In [16]:
for_resample_df

NameError: name 'for_resample_df' is not defined

In [17]:
#now resample all columns
resampled_time_series= []
appended_df = []

keys = ['ADLProperties_TotalUseTime','ADLProperties_NumberOfDeviceBoots',\
        'ADLProperties_NumberOfPowerDowns','ADLProperties_NumberOfPresetSwitches',\
        'VolumeControl']

cnt = 0
appended_df = []
for hi_id, group in for_resample_df.groupby('HearingInstrumentId'):
    cnt += 1
    if (cnt % 1000 == 0):
        print("Resampled " + str(cnt) + " ADLs")
    time_series = group    
    time_series = time_series.loc[~time_series.index.duplicated(keep='last')]
    time_series = time_series.sort_index()
    
    r_time_series = time_series[keys].diff().resample('W').sum().\
    interpolate(method = 'linear') 
        
    for colname, col in r_time_series.iteritems():
        bin = col ==0
        if bin.iloc[-1]==True:
            continue
        for i in np.arange(col.shape[0]):
            start_row = i
            replace_rows = []
            n = 0
            #print("Start row : " + str(start_row))
            while (bin[start_row] == True):
                replace_rows.append(start_row)
                start_row = start_row + 1
                n = n + 1
                #print("Replace rows : " + str(replace_rows))
            if len(replace_rows) > 0:
                bin.iloc[replace_rows] = False            
                replace_value = col.iloc[start_row] / (n+1)
                col.iloc[replace_rows] = replace_value
                col.iloc[start_row] = replace_value

    resampled_time_series = r_time_series / 7
    resampled_time_series['ADLRawData_HearingInstrumentId'] = hi_id
         
    appended_df.append(resampled_time_series)

new_resampled_df = pd.concat(appended_df)

NameError: name 'for_resample_df' is not defined

In [18]:
resampled_time_series

[]

In [19]:
new_resampled_df['class'] = 0
new_resampled_df.loc[new_resampled_df['ADLRawData_HearingInstrumentId'].isin(hi_id_last_ADL), 'class'] = 1

NameError: name 'new_resampled_df' is not defined

In [20]:
np.unique(new_resampled_df['class'])

NameError: name 'new_resampled_df' is not defined

In [21]:
#saving the resampled file
with open(base_path + 'resampled_last_ADL_before_return.pkl', 'wb') as handle:
    pickle.dump(new_resampled_df, handle)
    
with open(base_path + 'hi_id_ADL_before_return.pkl', 'wb') as handle:
    pickle.dump(hi_id_last_ADL, handle)

    

FileNotFoundError: [Errno 2] No such file or directory: '/home/tobiasz820/data/resampled_last_ADL_before_return.pkl'